In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
from tensorflow.keras.layers.experimental.preprocessing import *
import matplotlib.pyplot as plt
from hp import *

In [3]:
num_classes = 100
input_shape = (32, 32, 3)
checkpoint_path = "C:\\Users\\tom7l\\ckpoint"
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")


x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


In [4]:
cropped_image = tf.image.crop_to_bounding_box(x_train[0], 0, 0, 31, 32)
padded_image = tf.image.pad_to_bounding_box(cropped_image, 0, 0, 32, 32)

In [5]:
num_classes = 100
input_shape = (32, 32, 3)

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()
a=tf.keras.preprocessing.image.random_shift(
    x_train[0],1, 0, row_axis=1, col_axis=2, channel_axis=0,
    fill_mode='nearest', cval=0.0, interpolation_order=1
)
a=tf.keras.preprocessing.image.random_shift(
    x_train[0],15, 0, row_axis=1, col_axis=2, channel_axis=0,
    fill_mode='nearest', cval=0.0, interpolation_order=1
)


print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")


x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


In [6]:
data_augmentation = keras.Sequential(
    [
        Normalization(),
        Resizing(image_size, image_size),
        RandomFlip("horizontal"),
        RandomZoom(height_factor=0.2, width_factor=0.2),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(x_train)

In [7]:

def build_classifier(blocks, positional_encoding=False):
    inputs = layers.Input(shape=input_shape)
    

    # Augment data.
    augmented = data_augmentation(inputs)
    
    # Create patches.
    patches = Patches(patch_size, num_patches)(augmented)
    
    # Encode patches to generate a [batch_size, num_patches, embedding_dim] tensor.
    x = layers.Dense(units=embedding_dim)(patches)
    if positional_encoding:
        positions = tf.range(start=0, limit=num_patches, delta=1)
        position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=embedding_dim
        )(positions)
        x = x + position_embedding
    # Process x using the module blocks.
    x = blocks(x)
    # Apply global average pooling to generate a [batch_size, embedding_dim] representation tensor.
    representation = layers.GlobalAveragePooling1D()(x)
    # Apply dropout.
    representation = layers.Dropout(rate=dropout_rate)(representation)
    # Compute logits outputs.
    logits = layers.Dense(num_classes)(representation)
    # Create the Keras model.
    return keras.Model(inputs=inputs, outputs=logits)

In [8]:
  
def run_experiment(model):
    # Create Adam optimizer with weight decay.
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay,
    )
    # Compile the model.
    model.compile(
        optimizer=optimizer,

        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="acc"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top5-acc"),
        ],
    )
    # Create a learning rate scheduler callback.
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=5
    )
    # Create an early stopping callback.
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )
    # Fit the model.
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[early_stopping, reduce_lr,cp_callback],
    )

    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    # Return history to plot learning curves.
    return history


In [9]:
from classes import MLPMixerLayer, Patches
mlpmixer_blocks = keras.Sequential(
    [MLPMixerLayer(num_patches, embedding_dim, dropout_rate) for _ in range(num_blocks)]
)

model = build_classifier(mlpmixer_blocks)

# history = run_experiment(mlpmixer_classifier)

C:\Users\tom7l\anaconda3\envs\ai\lib\site-packages\tensorflow\python\autograph\impl\api.py:376: UserWarning: Default value of `approximate` is changed from `True` to `False`
  return py_builtins.overload_of(f)(*args)


In [10]:
# load trained model

model.load_weights(checkpoint_path)
# Create Adam optimizer with weight decay.
optimizer = tfa.optimizers.AdamW(
    learning_rate=learning_rate, weight_decay=weight_decay,
)
# Compile the model.
model.compile(
    optimizer=optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
        keras.metrics.SparseTopKCategoricalAccuracy(5, name="top5-acc"),
    ],
)
loss, acc, top5_acc = model.evaluate(x_test, y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

313/313 - 17s - loss: 1.8927 - acc: 0.5058 - top5-acc: 0.7952
Restored model, accuracy: 50.58%


In [24]:
from patch_test import *

# img = x_test[10]
# # img=crop_with_direction(img,bottom=1)
# # img=np.array(img)
# img=img.reshape((1,32,32,3))
# y_test[0].reshape((1,1))
# a=np.argmax(model.predict(img))
# predict_mat = model.predict(img)[0]
# # print(predict_mat)
# print(max(predict_mat))
# second_max_idx =  np.argpartition(predict_mat.flatten(), -2)[-2:]
# second_max_value = predict_mat[second_max_idx][0]
# print(second_max_value)
# # model.evaluate(img.reshape((1,32,32,3)),y_test[15],verbose=2)

def find_smallest_diffs(k):
    slice_object = slice(50)
    # diffs = [None] * x_test.size
    diffs = [None] * 50
    for i, image in enumerate(x_test[slice_object]):
        image = image.reshape((1,32,32,3))
        # max=np.argmax(model.predict(image))
        predict_matrix = model.predict(image)[0]
        max_value = max(predict_matrix)
        second_max_idx =  np.argpartition(predict_matrix.flatten(), -2)[-2:]
        second_max_value = predict_matrix[second_max_idx][0]
        diffs[i] = max_value-second_max_value
    print(diffs)
    slice_k = slice(k)
    smallest_images = np.argpartition(diffs,k)[slice_k]
    print(smallest_images)

find_smallest_diffs(5)


[0.6493306, 2.6064634, 0.94237137, 0.051816463, 3.9593754, 2.925706, 1.1266809, 0.4871168, 4.259143, 0.876956, 1.2505922, 3.5374303, 0.25746918, 7.3994026, 0.79383945, 2.7927084, 2.7273493, 0.34855556, 1.6368322, 1.5857592, 1.0949249, 1.2154498, 0.18168926, 0.5239978, 6.6065607, 2.1589203, 6.3039017, 4.675338, 1.2184777, 1.3687944, 3.8053799, 2.9740095, 2.4551196, 0.12508774, 2.702506, 0.76242495, 2.1097994, 0.82581806, 1.9268599, 0.31149578, 1.6002016, 0.096577644, 3.860211, 1.8792591, 7.0126657, 1.6900587, 0.81520367, 0.72397614, 4.0528383, 1.192544]
[ 3 41 33 22 12]
